In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns
import patsy
from sklearn.model_selection import train_test_split
%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
# Columns from kaggle dataset
old_col= '''Index(['Loan ID', 'Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')'''

# Read in CSV

In [3]:
#df = pd.read_csv('~/Downloads/LoanStats3c.csv',header = 1)
#df.to_pickle('./data/loan_data_2014.zip',compression = 'zip')

# Drop unnecessary columns:

In [4]:
df = pd.read_pickle('./data/loan_data_2014.zip',compression = 'zip')
df.shape

(235631, 145)

In [5]:
# Only take rows of loans that are charged off or fully paid as results
df = df.loc[(df['loan_status'] == 'Fully Paid') | (df['loan_status'] == 'Charged Off')]

In [6]:
df=df.drop(columns = [
        # Only looking at loans for individuals, not joint
        'annual_inc_joint',
        'application_type',
        'dti_joint',
        'verification_status_joint',
        'revol_bal_joint', 

        # Don't need specific ID info
        'id',
        'member_id',
    
        # Only care about information available pre loan
        'collection_recovery_fee',
        'collections_12_mths_ex_med',
        
        # Hard to classify, may use regex text processing in future
        'emp_title',
    
        # hardship info is post issuing
        'hardship_flag',
        'hardship_status',
        'hardship_type',
        'hardship_reason',
        'deferral_term',
        'hardship_amount',
        'hardship_start_date',
        'hardship_end_date',
        'payment_plan_start_date',
        'hardship_length',
        'hardship_dpd',
        'hardship_loan_status',
        'orig_projected_additional_accrued_interest',
        'hardship_payoff_balance_amount',
        'hardship_last_payment_amount',
    
        'last_pymnt_amnt',
        'last_pymnt_d',
        'next_pymnt_d',
        'pymnt_plan',
        'total_pymnt',
        'total_pymnt_inv',
        'total_rec_int',
        'total_rec_late_fee',
        'total_rec_prncp',
        # Trying to extrapolate beyond certain issuing dates of loans
        'issue_d',
        'mths_since_last_major_derog',
        'mths_since_last_record',
        'mths_since_recent_inq',
        'num_tl_30dpd',
        'num_tl_120dpd_2m',
        
        'debt_settlement_flag',
        'debt_settlement_flag_date',
        'desc',
        
        # sub_grade category already contains grade
        'grade',
    
        # This is for investors / loan funding
        'initial_list_status',
        'funded_amnt',
        'funded_amnt_inv',              
        # Probably useful metrics, but all NaN in this dataset              
        'open_acc_6m',                 
        'open_il_12m',
        'open_il_24m',                     
        'open_act_il',   
        'open_rv_12m',
        'open_rv_24m',
                      
        'out_prncp', 
        'out_prncp_inv',
        'policy_code',
        'recoveries',
        # Columns related to debt settlement; we only care about pre loan features              
        'settlement_status',
        'settlement_date',
        'settlement_amount',
        'settlement_percentage',
        'settlement_term',
        # Only looking for one applicant 
        #'sec_app_fico_range_low', 
        #'sec_app_fico_range_high', 
        'sec_app_earliest_cr_line', 
        'sec_app_inq_last_6mths', 
        'sec_app_mort_acc', 
        'sec_app_open_acc', 
        'sec_app_revol_util',
        'sec_app_open_act_il',
        'sec_app_num_rev_accts',
        'sec_app_chargeoff_within_12_mths',
        'sec_app_collections_12_mths_ex_med', 
        'sec_app_mths_since_last_major_derog', 
        'title',
        'total_cu_tl',
                      
        'url', 
        # May be good to look at later
        'zip_code'])

print('Before dropping NaN\'s: ',df.shape)
df.dropna(how='all', inplace = True)
print('After dropping NaN Rows: ',df.shape)
pre_drop_col = df.columns
df.dropna(axis=1, how='all',inplace = True)
print('After dropping NaN Columns: ',df.shape)
post_drop_col = df.columns

Before dropping NaN's:  (223102, 69)
After dropping NaN Rows:  (223102, 69)
After dropping NaN Columns:  (223102, 62)


In [7]:
dropped_nan_columns = set(pre_drop_col) - set(post_drop_col)
print("Dropped NaN Columns: ", dropped_nan_columns)

Dropped NaN Columns:  {'il_util', 'inq_fi', 'all_util', 'max_bal_bc', 'total_bal_il', 'inq_last_12m', 'mths_since_rcnt_il'}


In [8]:

#plt.figure(figsize=(24, 24))
#sns.heatmap(df.corr(),vmin = -1, vmax = 1, cmap = 'seismic')

In [9]:
df.sample(5)

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method
157648,5400.0,36 months,12.99%,181.93,C1,9 years,MORTGAGE,45000.0,Source Verified,Fully Paid,credit_card,UT,8.85,1.0,Nov-1998,2.0,19.0,10.0,0.0,4612.0,56.2%,28.0,May-2018,0.0,0.0,243052.0,8200.0,5.0,24305.0,588.0,88.7,0.0,0.0,182.0,186.0,5.0,5.0,4.0,5.0,NaN,19.0,1.0,6.0,6.0,6.0,14.0,7.0,8.0,17.0,6.0,10.0,0.0,4.0,92.9,83.3,0.0,0.0,257635.0,11640.0,5200.0,8325.0,Cash
29763,15000.0,36 months,13.66%,510.20,C3,4 years,MORTGAGE,90000.0,Source Verified,Fully Paid,debt_consolidation,DE,25.72,0.0,May-1993,0.0,36.0,21.0,0.0,27165.0,60.1%,30.0,Apr-2018,0.0,0.0,293595.0,45200.0,5.0,13981.0,7692.0,70.0,0.0,0.0,257.0,127.0,1.0,1.0,1.0,1.0,NaN,NaN,0.0,10.0,16.0,10.0,11.0,10.0,16.0,19.0,16.0,21.0,0.0,4.0,100.0,70.0,0.0,0.0,343853.0,123066.0,25600.0,107799.0,Cash
221424,16800.0,60 months,19.97%,444.82,D5,10+ years,MORTGAGE,250000.0,Verified,Fully Paid,other,NY,3.05,0.0,Feb-1984,4.0,60.0,9.0,0.0,21267.0,61.6%,28.0,Nov-2018,0.0,0.0,108406.0,34500.0,0.0,15487.0,12233.0,63.5,0.0,0.0,137.0,359.0,52.0,52.0,7.0,52.0,60.0,60.0,0.0,5.0,5.0,5.0,11.0,6.0,8.0,15.0,5.0,9.0,0.0,0.0,95.8,40.0,0.0,0.0,133500.0,21267.0,33500.0,0.0,Cash
105143,15000.0,36 months,9.17%,478.19,B1,6 years,MORTGAGE,75000.0,Not Verified,Fully Paid,debt_consolidation,PA,17.33,0.0,Dec-2002,2.0,NaN,17.0,0.0,13163.0,36.6%,36.0,Dec-2015,0.0,0.0,332165.0,36000.0,4.0,23726.0,10106.0,49.7,0.0,0.0,114.0,139.0,18.0,6.0,2.0,18.0,NaN,NaN,0.0,2.0,4.0,6.0,9.0,11.0,13.0,23.0,4.0,17.0,0.0,2.0,100.0,20.0,0.0,0.0,390054.0,35083.0,20100.0,54801.0,Cash
104907,5000.0,36 months,14.49%,172.09,C4,1 year,RENT,75000.0,Not Verified,Fully Paid,debt_consolidation,VA,7.94,0.0,Oct-1995,2.0,NaN,18.0,0.0,13910.0,89.2%,34.0,Mar-2017,0.0,0.0,138311.0,15600.0,8.0,7684.0,303.0,97.8,0.0,0.0,225.0,163.0,8.0,8.0,0.0,40.0,NaN,NaN,0.0,3.0,4.0,3.0,3.0,29.0,5.0,5.0,4.0,18.0,0.0,1.0,100.0,100.0,0.0,0.0,137656.0,138311.0,13500.0,122056.0,Cash


In [10]:
#df[df['num_tl_120dpd_2m'] >= 1][['num_tl_30dpd','num_tl_120dpd_2m','num_tl_90g_dpd_24m']]

In [11]:
# df[['int_rate','grade','sub_grade']].sort_values(by = 'int_rate')

# Clean Columns

In [12]:
def drop_percentage_signs(df, column_name):
    new_column_name = column_name + '_percent'
    df[new_column_name] = df[column_name].astype(str).str[:-1].astype(np.float64)
    return df.drop(columns = [column_name])

In [13]:
df.shape

(223102, 62)

In [14]:
#df['revol_util'].fillna('0%',inplace = True)
#df['revol_util'] =df['revol_util'].astype(str).str[:-1]


In [15]:
# Clean columns
# Convert loan terms into two categories 'short' and 'long'
df['term'] = df['term'].str.replace('36 months','short').str.replace('60 months','long').str.strip()

# Convert interest rate from percentage to float
df['int_rate'].dropna(axis = 0,how = 'any', inplace = True)
df=drop_percentage_signs(df,'int_rate')


# Fill Na's with 0.0% in revolving balance (unused revolving balance counts as NaN)
df['revol_util'].fillna(value = '0.0%',inplace = True)
df=drop_percentage_signs(df, 'revol_util')

In [16]:
df.shape

(223102, 62)

# One hot encoding 'Months Since' Variables

- First do a pd.cut() to bin the data into categories.
- Rename these during the function call
- Then fill nans as another category
- Run a pd.get_dummies on the resulting column

In [17]:
df['mths_since_last_delinq'] = \
pd.cut(df['mths_since_last_delinq'],5,labels =
       ['0-3 years','3-6 years','6-9 years','9-12 years','12-15 years'
       ]).cat.add_categories('never').fillna('never') # 

df['mths_since_recent_bc'] = \
pd.cut(df['mths_since_recent_bc'],5,labels =
       ['0-10 years','10-20 years','20-30 years','30-40 years','40-50 years']
      ).cat.add_categories('never').fillna('never')# 

df['mths_since_recent_bc_dlq'] = \
pd.cut(df['mths_since_recent_bc_dlq'],5, labels = 
      ['0-3 years','3-6 years','6-9 years','9-12 years','12-15 years'
       ]).cat.add_categories('never').fillna('never') # 

df['mths_since_recent_revol_delinq'] = \
pd.cut(df['mths_since_recent_revol_delinq'],5, labels = 
      ['0-3 years','3-6 years','6-9 years','9-12 years','12-15 years'
       ]).cat.add_categories('never').fillna('never') # 


In [18]:
#pd.get_dummies(df,drop_first = True)

In [19]:
def add_days_since_col(df,column_name):
    new_col_name = 'days_since_' + column_name
    df[new_col_name] = df[column_name].apply(lambda x: (dt.datetime.today().date() - 
                                                      dt.datetime.strptime(x,'%b-%Y').date()).days)
    return df.drop(columns = [column_name])
# df['days_since_earliest_cr_line'] = df['earliest_cr_line'].apply(lambda x: (dt.datetime.today().date() - 
#                                                       dt.datetime.strptime(x,'%b-%Y').date()).days)
#df=df.drop(columns = ['earliest_cr_line'])

df = add_days_since_col(df,'earliest_cr_line')
df= df.dropna(subset = ['last_credit_pull_d'])
df = add_days_since_col(df,'last_credit_pull_d')

In [20]:
#df = add_days_since_col(df,'last_credit_pull_d')

In [21]:
#df.lxoc[df['last_credit_pull_d'].isna()]

In [22]:
df.sample(5)

,loan_amnt,term,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,total_acc,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,int_rate_percent,revol_util_percent,days_since_earliest_cr_line,days_since_last_credit_pull_d
75636,6625.0,short,238.09,D4,2 years,RENT,137500.0,Verified,Charged Off,major_purchase,IN,20.41,2.0,1.0,0-3 years,18.0,3.0,1870.0,22.0,0.0,56.0,258982.0,4300.0,10.0,14388.0,1630.0,53.4,0.0,0.0,136.0,46.0,3.0,3.0,0.0,0-10 years,0-3 years,0-3 years,0.0,5.0,5.0,6.0,6.0,15.0,7.0,7.0,5.0,18.0,0.0,2.0,81.0,33.3,1.0,2.0,288964.0,258982.0,3500.0,284664.0,Cash,17.57,43.5,5842,909
9482,28000.0,long,674.83,D1,10+ years,MORTGAGE,110000.0,Source Verified,Fully Paid,credit_card,NY,12.47,0.0,1.0,never,8.0,0.0,22128.0,19.0,0.0,0.0,292638.0,23500.0,2.0,41805.0,1028.0,95.0,0.0,0.0,157.0,134.0,6.0,6.0,5.0,0-10 years,never,never,0.0,4.0,5.0,5.0,5.0,6.0,6.0,8.0,5.0,8.0,0.0,1.0,100.0,100.0,0.0,0.0,306708.0,70715.0,20500.0,57708.0,Cash,15.59,94.2,6419,59
155997,18900.0,short,645.78,C3,10+ years,RENT,45000.0,Source Verified,Fully Paid,debt_consolidation,NM,16.64,0.0,0.0,never,13.0,1.0,21292.0,19.0,0.0,0.0,21393.0,33300.0,3.0,1783.0,4729.0,78.5,0.0,0.0,132.0,210.0,5.0,5.0,0.0,0-10 years,never,never,0.0,6.0,11.0,6.0,9.0,1.0,12.0,17.0,11.0,13.0,0.0,2.0,100.0,50.0,0.0,0.0,33944.0,21393.0,22000.0,0.0,Cash,13.98,63.9,8214,483
109098,8000.0,short,261.88,B3,3 years,RENT,67600.0,Source Verified,Charged Off,credit_card,MO,17.13,0.0,2.0,never,20.0,2.0,14287.0,31.0,0.0,0.0,25981.0,32000.0,4.0,1299.0,12869.0,50.5,0.0,0.0,78.0,148.0,2.0,2.0,0.0,0-10 years,never,never,0.0,12.0,15.0,13.0,13.0,7.0,18.0,24.0,15.0,20.0,0.0,4.0,100.0,15.4,0.0,2.0,45317.0,25981.0,26000.0,13317.0,Cash,10.99,44.6,6268,302
169757,20000.0,short,635.07,A5,7 years,MORTGAGE,85000.0,Source Verified,Fully Paid,home_improvement,CA,10.18,0.0,1.0,never,13.0,0.0,6976.0,20.0,0.0,0.0,14653.0,29200.0,2.0,1127.0,18160.0,25.3,0.0,0.0,71.0,75.0,17.0,4.0,2.0,0-10 years,never,never,0.0,6.0,9.0,7.0,9.0,3.0,12.0,15.0,9.0,13.0,0.0,1.0,100.0,0.0,0.0,0.0,49215.0,14653.0,24300.0,20015.0,Cash,8.90,23.9,4136,848


# Patsy.dmatrix (pass string like last project, but no y~)

In [29]:
df.isnull().sum()

loan_amnt                             0
term                                  0
installment                           0
sub_grade                             0
emp_length                        11551
home_ownership                        0
annual_inc                            0
verification_status                   0
loan_status                           0
purpose                               0
addr_state                            0
dti                                   0
delinq_2yrs                           0
inq_last_6mths                        0
mths_since_last_delinq                0
open_acc                              0
pub_rec                               0
revol_bal                             0
total_acc                             0
acc_now_delinq                        0
tot_coll_amt                          0
tot_cur_bal                           0
total_rev_hi_lim                      0
acc_open_past_24mths                  0
avg_cur_bal                           6


In [34]:
df.loc[df['mo_sin_old_il_acct'].isna()].head()

,loan_amnt,term,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,total_acc,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,int_rate_percent,revol_util_percent,days_since_earliest_cr_line,days_since_last_credit_pull_d
27,18000.0,short,597.78,B5,8 years,OWN,70000.0,Not Verified,Fully Paid,debt_consolidation,FL,9.11,0.0,0.0,3-6 years,6.0,0.0,16644.0,13.0,0.0,0.0,111500.0,31800.0,2.0,18583.0,15156.0,52.3,0.0,0.0,NaN,162.0,8.0,8.0,3.0,0-10 years,never,3-6 years,0.0,4.0,4.0,4.0,6.0,0.0,4.0,9.0,4.0,6.0,0.0,1.0,92.3,25.0,0.0,0.0,143527.0,23192.0,31800.0,0.0,Cash,11.99,52.3,6541,544
124,6700.0,short,230.01,C4,2 years,RENT,21000.0,Not Verified,Fully Paid,credit_card,NY,19.94,0.0,0.0,never,5.0,0.0,10064.0,6.0,0.0,0.0,10064.0,15500.0,3.0,2013.0,5330.0,59.0,0.0,0.0,NaN,104.0,20.0,20.0,0.0,0-10 years,never,never,0.0,4.0,5.0,4.0,5.0,0.0,5.0,6.0,5.0,5.0,0.0,0.0,100.0,75.0,0.0,0.0,15500.0,10064.0,13000.0,0.0,Cash,14.31,64.9,4776,452
128,17000.0,short,567.82,C1,3 years,RENT,46300.0,Not Verified,Fully Paid,major_purchase,FL,5.81,0.0,1.0,never,4.0,0.0,10553.0,4.0,0.0,0.0,10553.0,15000.0,3.0,2638.0,2745.0,78.9,0.0,0.0,NaN,64.0,12.0,12.0,0.0,0-10 years,never,never,0.0,3.0,4.0,3.0,3.0,0.0,4.0,4.0,4.0,4.0,0.0,1.0,100.0,66.7,0.0,0.0,15000.0,10553.0,13000.0,0.0,Cash,12.39,70.4,3558,424
152,4000.0,short,125.70,A5,< 1 year,MORTGAGE,24000.0,Not Verified,Fully Paid,home_improvement,SD,10.00,0.0,0.0,never,9.0,3.0,7245.0,13.0,0.0,652.0,62585.0,22600.0,3.0,6954.0,8299.0,41.6,0.0,0.0,NaN,301.0,7.0,7.0,2.0,0-10 years,never,never,0.0,5.0,8.0,5.0,5.0,0.0,8.0,11.0,8.0,9.0,0.0,1.0,100.0,20.0,0.0,0.0,95600.0,7245.0,14200.0,0.0,Cash,8.19,32.1,10771,909
196,13825.0,long,328.83,C5,10+ years,MORTGAGE,33000.0,Not Verified,Charged Off,debt_consolidation,FL,11.78,0.0,0.0,never,11.0,0.0,13742.0,20.0,0.0,0.0,109137.0,33700.0,2.0,10914.0,14301.0,47.0,0.0,0.0,NaN,96.0,2.0,2.0,5.0,0-10 years,never,never,0.0,4.0,6.0,6.0,10.0,0.0,10.0,15.0,6.0,11.0,0.0,1.0,100.0,16.7,0.0,0.0,131400.0,13742.0,27000.0,0.0,Cash,14.99,40.8,8641,817


In [36]:
pd.cut(df['mo_sin_old_il_acct'],5).value_counts()

(100.2, 200.4]     146922
(-0.501, 100.2]     54171
(200.4, 300.6]      13997
(300.6, 400.8]       1000
(400.8, 501.0]         88
Name: mo_sin_old_il_acct, dtype: int64

In [41]:
df.loc[df['percent_bc_gt_75'].isna()]

,loan_amnt,term,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,total_acc,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,int_rate_percent,revol_util_percent,days_since_earliest_cr_line,days_since_last_credit_pull_d
25,5000.0,short,162.49,B3,4 years,MORTGAGE,52000.0,Source Verified,Charged Off,debt_consolidation,AZ,14.35,0.0,2.0,never,9.0,1.0,466.0,39.0,0.0,841.0,103332.0,5300.0,5.0,11481.0,NaN,NaN,0.0,0.0,137.0,161.0,1.0,1.0,4.0,never,never,never,0.0,0.0,2.0,0.0,4.0,14.0,4.0,20.0,2.0,9.0,0.0,2.0,100.0,NaN,1.0,0.0,118035.0,103332.0,0.0,112735.0,Cash,10.49,8.8,6511,271
69,15000.0,short,487.47,B3,10+ years,MORTGAGE,84000.0,Not Verified,Fully Paid,credit_card,AL,19.73,0.0,2.0,never,10.0,1.0,4590.0,77.0,0.0,0.0,80333.0,6600.0,12.0,8926.0,NaN,NaN,0.0,0.0,125.0,127.0,14.0,1.0,1.0,never,never,never,0.0,0.0,2.0,0.0,9.0,59.0,4.0,17.0,2.0,10.0,0.0,5.0,100.0,NaN,1.0,0.0,119505.0,46198.0,0.0,47305.0,Cash,10.49,69.5,5903,179
126,2400.0,short,84.96,D3,7 years,RENT,70000.0,Not Verified,Fully Paid,debt_consolidation,MI,10.61,5.0,0.0,0-3 years,16.0,0.0,1000.0,41.0,0.0,0.0,60646.0,1000.0,10.0,3790.0,NaN,NaN,0.0,0.0,174.0,169.0,29.0,2.0,0.0,never,0-3 years,0-3 years,0.0,0.0,1.0,0.0,6.0,24.0,1.0,17.0,1.0,16.0,1.0,4.0,82.9,NaN,0.0,0.0,59381.0,60646.0,0.0,58381.0,Cash,16.49,100.0,6906,149
162,1000.0,short,32.95,B4,7 years,MORTGAGE,83000.0,Not Verified,Fully Paid,medical,AZ,25.12,0.0,0.0,3-6 years,10.0,0.0,1605.0,44.0,0.0,0.0,132443.0,7900.0,5.0,14716.0,NaN,NaN,0.0,0.0,149.0,216.0,3.0,3.0,3.0,never,3-6 years,3-6 years,0.0,0.0,2.0,0.0,6.0,26.0,4.0,15.0,2.0,9.0,0.0,3.0,73.7,NaN,0.0,0.0,140458.0,132443.0,0.0,132558.0,Cash,11.44,20.3,8184,514
225,3075.0,short,112.01,E1,3 years,MORTGAGE,24000.0,Not Verified,Fully Paid,debt_consolidation,WA,33.70,0.0,0.0,3-6 years,8.0,1.0,7092.0,13.0,0.0,0.0,119085.0,18000.0,2.0,14886.0,NaN,NaN,0.0,0.0,60.0,84.0,2.0,2.0,1.0,never,3-6 years,3-6 years,0.0,0.0,6.0,0.0,2.0,4.0,6.0,8.0,6.0,8.0,0.0,1.0,92.3,NaN,0.0,1.0,141035.0,16286.0,0.0,15785.0,Cash,18.54,39.4,4167,1154
302,4800.0,short,164.78,C4,2 years,OWN,28500.0,Not Verified,Fully Paid,debt_consolidation,SC,7.12,0.0,0.0,0-3 years,2.0,0.0,394.0,9.0,0.0,0.0,172613.0,1500.0,0.0,86307.0,NaN,NaN,0.0,0.0,140.0,237.0,82.0,82.0,2.0,never,0-3 years,0-3 years,3.0,0.0,1.0,0.0,3.0,1.0,1.0,5.0,1.0,2.0,0.0,0.0,66.7,NaN,0.0,0.0,198086.0,394.0,0.0,0.0,Cash,14.31,26.3,8825,59
319,35000.0,short,1201.50,C4,2 years,RENT,140000.0,Verified,Fully Paid,debt_consolidation,VA,14.20,1.0,1.0,0-3 years,13.0,0.0,10372.0,48.0,0.0,0.0,77697.0,22700.0,5.0,8633.0,NaN,NaN,1.0,0.0,134.0,203.0,2.0,2.0,10.0,10-20 years,never,never,1.0,0.0,3.0,1.0,8.0,12.0,11.0,26.0,3.0,13.0,1.0,2.0,97.9,NaN,0.0,0.0,113156.0,77697.0,0.0,90456.0,Cash,14.31,45.7,7788,59
409,4800.0,short,178.37,E3,10+ years,MORTGAGE,67000.0,Not Verified,Fully Paid,credit_card,MD,13.40,0.0,0.0,never,6.0,0.0,12622.0,12.0,0.0,0.0,23461.0,16200.0,2.0,3910.0,NaN,NaN,0.0,0.0,43.0,120.0,10.0,6.0,2.0,never,never,never,0.0,0.0,3.0,0.0,0.0,2.0,5.0,8.0,3.0,6.0,0.0,2.0,100.0,NaN,0.0,0.0,28200.0,23461.0,0.0,12000.0,Cash,19.99,77.9,6268,483
877,2000.0,short,68.03,C3,10+ years,MORTGAGE,72000.0,Not Verified,Fully Paid,vacation,TX,34.28,0.0,0.0,never,7.0,0.0,9042.0,28.0,0.0,0.0,258451.0,10500.0,6.0,36922.0,NaN,NaN,0.0,0.0,124.0,1

In [42]:
df.shape

(223078, 62)

In [57]:
df_temp = df.dropna()

In [62]:
def patsy_string_from_df(df):
    df = df.drop(columns = ['loan_status'])
    patsy_string = ''
    for title in df.columns:
        patsy_string += title + " + "
    #remove last plus and spaces from string
    patsy_string=patsy_string[:-3]
    print("Patsy String: " +patsy_string)
    return patsy_string

In [63]:
patsy_string_from_df(df_temp)

Patsy String: loan_amnt + term + installment + sub_grade + emp_length + home_ownership + annual_inc + verification_status + purpose + addr_state + dti + delinq_2yrs + inq_last_6mths + mths_since_last_delinq + open_acc + pub_rec + revol_bal + total_acc + acc_now_delinq + tot_coll_amt + tot_cur_bal + total_rev_hi_lim + acc_open_past_24mths + avg_cur_bal + bc_open_to_buy + bc_util + chargeoff_within_12_mths + delinq_amnt + mo_sin_old_il_acct + mo_sin_old_rev_tl_op + mo_sin_rcnt_rev_tl_op + mo_sin_rcnt_tl + mort_acc + mths_since_recent_bc + mths_since_recent_bc_dlq + mths_since_recent_revol_delinq + num_accts_ever_120_pd + num_actv_bc_tl + num_actv_rev_tl + num_bc_sats + num_bc_tl + num_il_tl + num_op_rev_tl + num_rev_accts + num_rev_tl_bal_gt_0 + num_sats + num_tl_90g_dpd_24m + num_tl_op_past_12m + pct_tl_nvr_dlq + percent_bc_gt_75 + pub_rec_bankruptcies + tax_liens + tot_hi_cred_lim + total_bal_ex_mort + total_bc_limit + total_il_high_credit_limit + disbursement_method + int_rate_percent

'loan_amnt + term + installment + sub_grade + emp_length + home_ownership + annual_inc + verification_status + purpose + addr_state + dti + delinq_2yrs + inq_last_6mths + mths_since_last_delinq + open_acc + pub_rec + revol_bal + total_acc + acc_now_delinq + tot_coll_amt + tot_cur_bal + total_rev_hi_lim + acc_open_past_24mths + avg_cur_bal + bc_open_to_buy + bc_util + chargeoff_within_12_mths + delinq_amnt + mo_sin_old_il_acct + mo_sin_old_rev_tl_op + mo_sin_rcnt_rev_tl_op + mo_sin_rcnt_tl + mort_acc + mths_since_recent_bc + mths_since_recent_bc_dlq + mths_since_recent_revol_delinq + num_accts_ever_120_pd + num_actv_bc_tl + num_actv_rev_tl + num_bc_sats + num_bc_tl + num_il_tl + num_op_rev_tl + num_rev_accts + num_rev_tl_bal_gt_0 + num_sats + num_tl_90g_dpd_24m + num_tl_op_past_12m + pct_tl_nvr_dlq + percent_bc_gt_75 + pub_rec_bankruptcies + tax_liens + tot_hi_cred_lim + total_bal_ex_mort + total_bc_limit + total_il_high_credit_limit + disbursement_method + int_rate_percent + revol_util

In [65]:
patsy.dmatrix(patsy_string_from_df(df_temp),df_temp)

Patsy String: loan_amnt + term + installment + sub_grade + emp_length + home_ownership + annual_inc + verification_status + purpose + addr_state + dti + delinq_2yrs + inq_last_6mths + mths_since_last_delinq + open_acc + pub_rec + revol_bal + total_acc + acc_now_delinq + tot_coll_amt + tot_cur_bal + total_rev_hi_lim + acc_open_past_24mths + avg_cur_bal + bc_open_to_buy + bc_util + chargeoff_within_12_mths + delinq_amnt + mo_sin_old_il_acct + mo_sin_old_rev_tl_op + mo_sin_rcnt_rev_tl_op + mo_sin_rcnt_tl + mort_acc + mths_since_recent_bc + mths_since_recent_bc_dlq + mths_since_recent_revol_delinq + num_accts_ever_120_pd + num_actv_bc_tl + num_actv_rev_tl + num_bc_sats + num_bc_tl + num_il_tl + num_op_rev_tl + num_rev_accts + num_rev_tl_bal_gt_0 + num_sats + num_tl_90g_dpd_24m + num_tl_op_past_12m + pct_tl_nvr_dlq + percent_bc_gt_75 + pub_rec_bankruptcies + tax_liens + tot_hi_cred_lim + total_bal_ex_mort + total_bc_limit + total_il_high_credit_limit + disbursement_method + int_rate_percent

DesignMatrix with shape (203151, 179)
  Columns:
    ['Intercept',
     'term[T.short]',
     'sub_grade[T.A2]',
     'sub_grade[T.A3]',
     'sub_grade[T.A4]',
     'sub_grade[T.A5]',
     'sub_grade[T.B1]',
     'sub_grade[T.B2]',
     'sub_grade[T.B3]',
     'sub_grade[T.B4]',
     'sub_grade[T.B5]',
     'sub_grade[T.C1]',
     'sub_grade[T.C2]',
     'sub_grade[T.C3]',
     'sub_grade[T.C4]',
     'sub_grade[T.C5]',
     'sub_grade[T.D1]',
     'sub_grade[T.D2]',
     'sub_grade[T.D3]',
     'sub_grade[T.D4]',
     'sub_grade[T.D5]',
     'sub_grade[T.E1]',
     'sub_grade[T.E2]',
     'sub_grade[T.E3]',
     'sub_grade[T.E4]',
     'sub_grade[T.E5]',
     'sub_grade[T.F1]',
     'sub_grade[T.F2]',
     'sub_grade[T.F3]',
     'sub_grade[T.F4]',
     'sub_grade[T.F5]',
     'sub_grade[T.G1]',
     'sub_grade[T.G2]',
     'sub_grade[T.G3]',
     'sub_grade[T.G4]',
     'sub_grade[T.G5]',
     'emp_length[T.10+ years]',
     'emp_length[T.2 years]',
     'emp_length[T.3 years]',
   